In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.utils import shuffle
import os
from PIL import Image  
import time

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras import optimizers
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.losses import CategoricalCrossentropy

%matplotlib inline

In [5]:
def load_data_numpy(dir):
    data_dir = os.path.join(dir, "data.npy") 
    label_dir = os.path.join(dir, "label.npy") 
    return np.load(data_dir), np.load(label_dir)

In [4]:
def load_data_image(dir):
    LABEL_LIST = [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    X = []
    y_onehot = np.zeros((60000, 10))
    
    for num in range(10):
        sub_dir = os.path.join(dir, str(LABEL_LIST[num]))
        for img_dir in os.listdir(sub_dir):
            img = Image.open(os.path.join(sub_dir, img_dir))
            array_img = np.array(img)
            X.append(np.array(img))
        
        y_onehot[6000*(num):6000*(num+1), num] = 1
    
        # Show example of number
        # img.show(title=str(num+1))
        plt.figure()
        plt.title(str(LABEL_LIST[num]))
        plt.imshow(array_img)
        plt.show()
    
    X = np.reshape(np.array(X), (60000, 28, 28, 1))
    X_shuffle, y_shuffle = shuffle(X, y_onehot, random_state=1)
    
    # np.save('data.npy', X_shuffle)
    # np.save('label.npy', y_shuffle)
    
    return X_shuffle, y_shuffle

In [6]:
drive_folder = "/content/drive/My Drive/Colab Notebooks/training_data" 
X, y = load_data_numpy(drive_folder)
#my_folder = "training_data_image"
#X, y = load_data_image(my_folder)

# Splitting dataset

In [6]:
X_train = X[:50000, :, :] 
y_train = y[:50000, :] 

X_small = X_train[:1000, :, :]
y_small = y[:1000, :] 

X_test = X[50000:, :, :] 
y_test = y[50000:, :] 

In [ ]:
model_ann = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(500, activation='relu'),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])
opt_ann = keras.optimizers.Adam(1e-3)
model_ann.compile(optimizer=opt_ann, loss=CategoricalCrossentropy(), metrics=['accuracy'])
model_ann.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_17 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 500)               392500    
_________________________________________________________________
dense_47 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_48 (Dense)             (None, 10)                1010      
Total params: 443,610
Trainable params: 443,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ann.fit(X_small, y_small, epochs=50, verbose=2)
test_loss, tess_acc = model_ann.evaluate(X_test, y_test)
print("Loss:", test_loss, "Accuracy:", tess_acc)

Epoch 1/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 2/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 3/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 4/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 5/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 6/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 7/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 8/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 9/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 10/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 11/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 12/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 13/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 14/50
32/32 - 0s - loss: 2.3009 - accuracy: 0.1150
Epoch 15/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 16/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 17/50
32/32 - 0s - loss: 2.3009 - accuracy: 0.1150
Epoch 18/50
32/32 - 0s - loss: 2.3008 - 

# CNN

In [10]:
def build_CNN(reg):
    model_cnn = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(),
        #Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(),
        #Dropout(0.25),
        
        Flatten(),
        Dense(512, activation='relu'),
        #Dropout(0.5),
        Dense(10, activation='softmax', activity_regularizer=l2(reg))
    ])
    #Adam, batch_size, #nesterov=True, momentum=0.9

    opt_cnn = optimizers.Adam(learning_rate=1e-4)
    # good learning_rate = 1e-4
    model_cnn.compile(optimizer=opt_cnn, loss=CategoricalCrossentropy(), metrics=['accuracy'])
    model_cnn.summary()

    return model_cnn

In [14]:
model_cnn = build_CNN(1e-4)
model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=1)

Epoch 1/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.6771 - accuracy: 0.7709 - val_loss: 0.1942 - val_accuracy: 0.9376
Epoch 2/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.1078 - accuracy: 0.9666 - val_loss: 0.1271 - val_accuracy: 0.9589
Epoch 3/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0444 - accuracy: 0.9868 - val_loss: 0.0985 - val_accuracy: 0.9661
Epoch 4/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0236 - accuracy: 0.9936 - val_loss: 0.0739 - val_accuracy: 0.9764
Epoch 5/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0164 - accuracy: 0.9954 - val_loss: 0.0430 - val_accuracy: 0.9860
Epoch 6/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0120 - accuracy: 0.9965 - val_loss: 0.0654 - val_accuracy: 0.9774
Epoch 7/200
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0100 - accuracy: 0.9970 - val_loss: 0.0639 - val_ac

# Test

In [ ]:
model_cnn.predict()

# Tuning parameters

In [16]:
reg_list = {}
for i in range(15): 
    if i<5:
        reg = np.random.uniform(5e-1, 1e-1)
    elif i<10:
        reg = np.random.uniform(1e-1, 5e-2)
    else:
        reg = np.random.uniform(5e-2, 1e-2)
    
    model_cnn = build_CNN(reg)
    history = model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=0)  # epochs=150-200 is good
    reg_list[reg] = [history.history['loss'][-1], history.history['val_accuracy'][-1]]
    print(i, reg, reg_list[reg])

0 0.25510366568249887 [0.280549019575119, 0.9822499752044678]
----- 845.1912641525269 -----
1 0.3762415828581967 [0.39725378155708313, 0.9828333258628845]
----- 838.0196979045868 -----
2 0.4756743098754319 [0.49396318197250366, 0.9787499904632568]
----- 832.0010342597961 -----
3 0.462460148192697 [0.48272037506103516, 0.981166660785675]
----- 831.6589376926422 -----
4 0.14740726588279274 [0.17077137529850006, 0.9837499856948853]
----- 835.4409701824188 -----
5 0.05373910570040294 [0.07734449952840805, 0.9858333468437195]
----- 841.1221489906311 -----
6 0.09236392113668954 [0.11935978382825851, 0.9839166402816772]
----- 844.0579044818878 -----
7 0.09684394455249037 [0.11932005733251572, 0.9872499704360962]
----- 850.6275310516357 -----
8 0.07407881737818325 [0.09971542656421661, 0.9831666946411133]
----- 847.9910509586334 -----
9 0.07016178656440046 [0.09271429479122162, 0.9863333106040955]
----- 843.3920969963074 -----
10 0.045631979544654394 [0.06801752001047134, 0.984666645526886]
--

In [ ]:
reg_list

**Comment:** 

*   Training accuracy reach 100% after 150 epoches with loss at 0.005.
*   Changing regularlization doesn't change validation accuracy much and it is still platue at 0.96 after 100 epoches with loss at 0.13 and then loss increase back to 0.15. 


# Saving model

In [10]:
starttime = time.time()
print(time.time() - starttime)
# import h5py
# model_cnn.save("cnn_model.h5")

4.649162292480469e-05


# Loading model to run on test set

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model("cnn_model.h5")
new_model.summary()

In [ ]:
# test = 
# new_model.fit(test)